In [2]:
model = create_model("EVA_AltCLIP_g_14","/sharefs/baai-mrnd/sunquan/eva_clip_test.pt")

<All keys matched successfully>


In [1]:
from clip.eva_clip import create_model
import torch

model = create_model("EVA_AltCLIP_g_14","/sharefs/baai-mrnd/sunquan/eva_clip_psz14.pt")
model.text.load_state_dict(torch.load("/home/chenzhongzhi/save_ckpt/kd_9lgs_18m_eva_debug/hf_encoder.pt",map_location='cpu'))
torch.save(model.state_dict(),"/sharefs/baai-mrnd/sunquan/eva_clip_test.pt")

_IncompatibleKeys(missing_keys=['text.proj', 'text.transformer.roberta.embeddings.position_ids', 'text.transformer.roberta.embeddings.word_embeddings.weight', 'text.transformer.roberta.embeddings.position_embeddings.weight', 'text.transformer.roberta.embeddings.token_type_embeddings.weight', 'text.transformer.roberta.embeddings.LayerNorm.weight', 'text.transformer.roberta.embeddings.LayerNorm.bias', 'text.transformer.roberta.encoder.layer.0.attention.self.query.weight', 'text.transformer.roberta.encoder.layer.0.attention.self.query.bias', 'text.transformer.roberta.encoder.layer.0.attention.self.key.weight', 'text.transformer.roberta.encoder.layer.0.attention.self.key.bias', 'text.transformer.roberta.encoder.layer.0.attention.self.value.weight', 'text.transformer.roberta.encoder.layer.0.attention.self.value.bias', 'text.transformer.roberta.encoder.layer.0.attention.output.dense.weight', 'text.transformer.roberta.encoder.layer.0.attention.output.dense.bias', 'text.transformer.roberta.enc

In [7]:
from transformers import AutoModel
model = AutoModel.from_pretrained("bert-base-uncased")

model_p = sum(p.numel() for p in model.parameters() if p.requires_grad)
(model_p ) >> 20 # Million

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


104

In [ ]:
# load model from KDmodel checkpoint.
from open_source_models.modeling_altclip import AltCLIP
from open_source_models.processing_altclip import AltCLIPProcessor
from open_source_models.modeling_kd import KDmodel
from transformers import PretrainedConfig
import torch
model_path = '/home/chenzhongzhi/save_ckpt/9lg_mlm'

config = PretrainedConfig.from_json_file(model_path + "/config.json")
kd = KDmodel.from_pretrained(model_path,config=config)
model = AltCLIP.from_pretrained("/home/chenzhongzhi/save_ckpt/xlm1024-9lg-18m_ft")
model_p = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
(model_p ) >> 20 # Million

825

In [2]:
# load model from KDmodel checkpoint.
from open_source_models.modeling_altclip import AltCLIP
from open_source_models.processing_altclip import AltCLIPProcessor
from open_source_models.modeling_kd import KDmodel
from transformers import PretrainedConfig
import torch
model_path = '/home/chenzhongzhi/save_ckpt/9lg_mlm'

config = PretrainedConfig.from_json_file(model_path + "/config.json")
config.add_lm_task=True
config.eva=False
kd = KDmodel.from_pretrained(model_path,config=config)
model = AltCLIP.from_pretrained("/home/chenzhongzhi/save_ckpt/xlm1024-9lg-18m_ft")
processor = AltCLIPProcessor.from_pretrained("/home/chenzhongzhi/save_ckpt/xlm1024-9lg-18m_ft")
model.text_model = kd.student
model.config.text_config = model.text_model.config
# model.load_state_dict(torch.load(model_path + "/xlmr_9lg_clip_18m_ft1e.pt",map_location='cpu'),strict=True)
model.save_pretrained("/home/chenzhongzhi/save_ckpt/9lg_lmloss")
processor.save_pretrained("/home/chenzhongzhi/save_ckpt/9lg_lmloss")


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.8.layer_norm1.bias', 'vision_model.encoder.layers.16.mlp.fc2.weight', 'vision_model.encoder.layers.12.layer_norm2.weight', 'vision_model.encoder.layers.16.mlp.fc1.weight', 'vision_model.encoder.layers.11.self_attn.k_proj.weight', 'vision_model.encoder.layers.4.mlp.fc2.bias', 'vision_model.encoder.layers.12.self_attn.out_proj.weight', 'vision_model.encoder.layers.2.layer_norm2.bias', 'vision_model.encoder.layers.18.mlp.fc2.bias', 'vision_model.encoder.layers.12.self_attn.v_proj.weight', 'vision_model.encoder.layers.19.layer_norm1.bias', 'vision_model.encoder.layers.22.mlp.fc2.weight', 'vision_model.encoder.layers.15.layer_norm2.weight', 'vision_model.encoder.layers.16.self_attn.out_proj.bias', 'vision_model.encoder.layers.9.self_attn.q_proj.bias', 'vision_model.encoder.layers.4.mlp.fc2.weight', 'vision_model.encoder.layers.18.mlp.fc1.bias', '

In [2]:
import torch
from clip import build_eva_model_and_transforms,tokenize
from PIL import Image

eva_clip_path = "/sharefs/baai-mrnd/sunquan/eva_clip_psz14.pt"
model_name = "EVA_CLIP_g_14"
image_path = "clip/some_image.jpg"
caption = ["a diagram", "a dog", "a cat"]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = build_eva_model_and_transforms(model_name, pretrained=eva_clip_path)

image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
text = tokenize(caption).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

<All keys matched successfully>


ValueError: too many values to unpack (expected 2)

In [2]:
# extract text encoder from EVA
from clip.eva_clip import create_model
model = create_model("EVA_CLIP_g_14","/sharefs/baai-mrnd/sunquan/eva_clip_psz14.pt")



from PIL import Image
import requests

# transformers version >= 4.21.0
from modeling_altclip import AltCLIP
from processing_altclip import AltCLIPProcessor

# now our repo's in private, so we need `use_auth_token=True`
model = AltCLIP.from_pretrained("BAAI/AltCLIP")
processor = AltCLIPProcessor.from_pretrained("BAAI/AltCLIP")
processor.tokenizer([text...])

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities


<All keys matched successfully>


In [4]:
import torch
torch.save(model.text,"/home/chenzhongzhi/ckpt/EVA_text.pt")
# model.text.

In [ ]:
def quickSort(left,right,nums):
    # find the partition
    i = left - 1
    key = nums[right]
    for j in range(left,right):
        if nums[j] < key:
            i += 1
            nums[i],nums[j] = nums[j],nums[i]
    idx = i+1 
    nums[idx],nums[right] = nums[right],nums[idx]
    quickSort(left,idx-1,nums)
    quickSort(idx+1,right,nums)
    
    
nums = [1,2,3,31,1,1,2,6,7]

quickSort(0,len(nums),nums)
    

: 